In [72]:
%run generate_matrices_tools.ipynb

[ 2 -2]
[ 0  1] :  B
[ 8 -2]
[-2  1] :  G
(0, 1) :  lincomb_LLL
(0.25000, 1.0000) :  lincomb_cube
(0, 1) :  sv_LLL
(0.50000, 0.50000) :  sv_cube
(-0.25000, 0.00000) :  lincomb_diff
[0.5 0.0]
[0.0 0.0] :  Diff impact

[ 1  2]
[-2 -2] :  B
[ 5 -6]
[-6  8] :  G
(1, 1) :  lincomb_LLL
(1.2000, 1.0000) :  lincomb_cube
(-1, 0) :  sv_LLL
(-0.80000, 0.40000) :  sv_cube
(-0.20000, 0.00000) :  lincomb_diff
[0.2 0.0]
[0.0 0.0] :  Diff impact

[-1  2]
[ 1  2] :  B
[5 3]
[3 5] :  G
(-1, 1) :  lincomb_LLL
(-0.60000, 1.0000) :  lincomb_cube
(2, 0) :  sv_LLL
(1.6000, 0.80000) :  sv_cube
(-0.40000, 0.00000) :  lincomb_diff
[ 0.8 -0.0]
[-0.0  0.0] :  Diff impact

[1 0]
[1 2] :  B
[1 1]
[1 5] :  G
(1, 0) :  lincomb_LLL
(1.0000, -0.20000) :  lincomb_cube
(1, 0) :  sv_LLL
(0.80000, -0.40000) :  sv_cube
(0.00000, 0.20000) :  lincomb_diff
[0.0 0.0]
[0.0 0.2] :  Diff impact

[2 0]
[0 2] :  B
[4 0]
[0 4] :  G
(0, 1) :  lincomb_LLL
(0.00000, 1.0000) :  lincomb_cube
(0, 2) :  sv_LLL
(0.00000, 2.0000) :  sv_cube
(

In [73]:
# returns matrix mu and list of squared euclidean norms
#   of the vectors in Gram-Schmidt orthogonalized basis
#   (Cholesky decomposition avoiding use of irrationals)
# G ... Gram matrix to be decomposed
def chol_decomp(G, cols=-1):
    rows = G.nrows()
    if cols < 0:
        cols = rows
    mu = matrix(QQ, rows, cols)
    squares = vector(QQ, cols)
    for y in range(rows):
        over_cols = (y>=cols)
        for x in range(cols if over_cols else y):
            t = 0
            for i in range(x):
                t += mu[y,i]*mu[x,i]*squares[i]
            mu[y,x] = (G[y,x]-t)/squares[x]
        if not over_cols:
            t = 0
            for i in range(y):
                t += mu[y,i]*mu[y,i]*squares[i]
            mu[y,y] = 1
            squares[y] = G[y,y]-t
    return (mu, squares)

# auxiliary function: swaps i-th row with the previous one
def swap_rows_in_mu(mu, B, i, n):
    mu[i-1], mu[i] = mu[i], mu[i-1]
    mu[i,i], mu[i-1,i] = 1, 0
    r = mu[i-1,i-1]
    mu[i-1,i-1] = 1
    temp = B[i-1]*r**2 + B[i]
    mu[i,i-1] = r*B[i-1]/temp
    B[i] = B[i-1]*B[i]/temp
    B[i-1] = temp
    for j in range(i+1, n):
        mu[j,i-1], mu[j,i] = mu[i,i-1]*mu[j,i-1]+mu[j,i]*(1-r*mu[i,i-1]), \
                           mu[j,i-1]-r*mu[j,i]

# auxiliary function: the inner cycle of LLL
def LLL_reduce_row(b, mu, i, h):
    for j in range(h-1, -1, -1):
        t = round(mu[i, j])
        for k in range(j+1):
            mu[i, k] -= t*mu[j, k]
        b[i] -= t*b[j]

# auxiliary function: LLL reduction algorithm
# b ... integer basis (does'n have to be square matrix)
# mu ... Gram-Schmidt mu matrix (doesn't have to be square matrix)
# B ... squared norms of Gram-Schmidt orthogonalized vectors
# n ... number of rows of b to be reduced (usually b.nrows())
# h ... index of first row not to be swapped (only reduced)
def LLL_aux(b, delta, mu, B, n, h):
    i = 1
    flag = True
    swaps_data = []
    while i < h:
        if flag:
            LLL_reduce_row(b, mu, i, i)
        if i > 0 and (delta-mu[i,i-1]**2)*B[i-1] > B[i]:
            b[i], b[i-1] = b[i-1], b[i]
            swap_rows_in_mu(mu, B, i, n)
            swaps_data.append([True, shortestVector(b).norm().n(digits=4)]) 

            i -= 1

            flag = False
        else:
            i += 1
            flag = True
            swaps_data.append([False, shortestVector(b)[0]])
    while i < n:  # performed only if n>h
        LLL_reduce_row(b, mu, i, h)
        i += 1
    return b, mu, B, swaps_data

# classical LLL algorithm
# last row of b can be used as a target point in Babai's algorithm
def LLL_own(b, delta=3/4, babai_bool=False):
    mu, B = chol_decomp(b*b.transpose(), b.nrows()-babai_bool)
    LLL = LLL_aux(copy(b), delta, mu, B, b.nrows(), b.nrows()-babai_bool)
    return LLL[3]

# LLL for triangular input basis
def LLL_triangular_input(b, delta=3/4, babai_bool=False):
    B = vector(QQ, [b[j,j]**2 for j in range(b.ncols())])
    mu = matrix(QQ, b)
    for j in range(mu.ncols()):
        r = mu[j,j]
        for i in range(j, mu.nrows()):
            mu[i,j] /= r
    return LLL_aux(copy(b), delta, mu, B, b.nrows(), b.nrows()-babai_bool)[0]


In [74]:
swapdata = LLL_own(randomMatrix(10, 50),delta=0.99)

for boolean, length in swapdata:
    if boolean:
        print(length)
    else:
        print("--")

--
54.04
--
--
54.04
--
--
54.04
54.04
--
--
--
54.04
54.04
--
--
--
54.04
54.04
54.04
54.04
--
--
--
--
54.04
--
--
54.04
54.04
54.04
--
--
54.04
--
--
--
54.04
54.04
54.04
54.04
54.04
--
--
--
--
54.04
--
--
--
41.18
41.18
41.18
41.18
41.18
41.18
41.18
41.18
41.18
--
--
--
--
--
--
--
--
41.18
41.18
41.18
41.18
41.18
--
--
41.18
--
--
--
--
--
38.96
38.96
38.96
38.96
38.96
38.96
38.96
38.96
38.96
--
--
--
--
38.96
38.96
38.96
--
--
--
--
--
38.96
--
--
38.96
38.96
38.96
38.96
--
--
--
--
--
--
38.96
38.96
--
--
38.96
--
--
